In [1]:
from sql_engine import connect
import datetime
import pandas as pd

In [2]:
def check_diff_days(diff_day):
    today = datetime.datetime.today()
    diff_day = datetime.timedelta(days=diff_day)
    prev_day = today-diff_day
    interval = today-prev_day
    days = interval.days
    begin_signal = datetime.datetime(2020, 10, 16)
    if prev_day<begin_signal:
        return (today-begin_signal).days
    else:
        return days

In [51]:
def load_yx_balance_apply_data(diff_days_start, diff_days_end, yx_con):
    diff_days_start = check_diff_days(diff_days_start)
    diff_days_end = check_diff_days(diff_days_end)
    balance_apply_SQL = f'''
	
SELECT
		CAST(
		t1.id AS CHAR ( 30 )) id,
		t1.trans_time,
		LEFT ( t1.trans_time, 10 ) create_date,
		t1.trans_no SourceNo,
		CAST(
		t3.user_id AS CHAR ( 30 )) user_id,
		t3.union_id UnionNumber,
		t3.name name,
		t2.out_id activityId,
        t2.policy_name,
		t1.title,
		t1.body,
	IF
		( t1.amt_trans > 0, t1.amt_trans / 100, 0 ) amt_in,
	IF
		( t1.amt_trans < 0, t1.amt_trans / 100, 0 ) amt_out,
	IF
		( t1.amt_trans > 0, t1.amt_trans / 100, 0 ) +
	IF
		( t1.amt_trans < 0, t1.amt_trans / 100, 0 ) money
	FROM
		oryx_account.account_trans_record t1
		INNER JOIN oryx_policy.policy t2 ON t1.policy_id = t2.policy_id
		INNER JOIN oryx_agent.agent_info t3 ON t1.customer_id = t3.user_id
		left join oryx_bi.zclx t6 on t2.out_id = t6.zc_id
		
	WHERE
		t1.company_id IN ( 12, 979 ) 
		AND t6.ActivityCWTypeNo in (1,2)
		and t2.state = 1
		AND t1.customer_type = 0 
		AND t1.account_type = 0 
		AND ( t1.type NOT IN ( 'pay', 'refund' ) OR t1.title = '余额申请单' ) 
		AND t1.title NOT IN ( '历史订单取消发货单余额退款', '车票审核补报上账', '历史订单余额退款', '车票审核报销上账' ) 
		AND DATEDIFF( NOW(), t1.trans_time )<={diff_days_start} AND DATEDIFF(NOW(), t1.trans_time)>={diff_days_end}
		AND t1.policy_id IS NOT NULL 
		AND ( t1.remark NOT LIKE '%%测试%%' OR t1.remark IS NULL ) 
		and title = '余额申请单'
		AND t3.union_id not in ('CM66381516','CM00006317','CM00007650')
                                   '''
    balance_apply_data = pd.read_sql(balance_apply_SQL, yx_con)
    return balance_apply_data

In [54]:
def load_erp_balance_apply_data(diff_days_start, diff_days_end, erp_con):
    diff_days_start = check_diff_days(diff_days_start)
    diff_days_end = check_diff_days(diff_days_end)
    balance_apply_SQL = f'''SELECT
-- 	iif (a.PaymentAccount = '','无账号',a.PaymentAccount) merchant_id,
	ISNULL( auditime, a.createTime ) trans_time,
	CONVERT(varchar(100), ISNULL( auditime, a.createTime ), 23) audi_date,
	IIF (
		( a.SourceNo IS NULL OR LEN( a.SourceNo ) = 0 ) 
		AND a.OrderNo LIKE 'SCBA%',
		a.OrderNo,
		a.SourceNo 
	) SourceNo,
	IIF (
		a.agentId= 'dab70acc-9b11-44e9-bc95-d6ad1e573336',
		'CM00000035',
		ISNULL(
			IIF (
				b.UnionNumber= 'CM88810886',
				'CM00020001',
				IIF (
					b.UnionNumber= 'CM00007497',
					'CM00007482',
					IIF (
						b.UnionNumber= 'CM00007595',
						'CM00007113',
						IIF ( b.UnionNumber= 'CM00006816', 'CM00006194', IIF ( b.UnionNumber= 'CM00007136', 'CM00000304', b.UnionNumber ) ) 
					) 
				) 
			),
			c.UnionNumber 
		) 
	) UnionNumber,
	isnull(c.realname,b.RealName) realname,
	d.zcname,
	item,
	InValue,
	- OutValue OutValue,
	InValue - OutValue money
FROM
	SCLMERPDB_UE.dbo.DH_BalanceApply a
	LEFT JOIN SCLM_DRPDB.dbo.b_UserInfo b ON a.agentId= b.f_id
	LEFT JOIN SCLM_DRPDB.dbo.b_UnionUserInfo c ON b.UnionNumber= c.UnionNumber 
	AND isdelete = 0
	LEFT JOIN dbo.ZCLX d ON a.activityid = d.zc_id 
WHERE
	a.status = 1 
	AND Deleted = 0 
	AND datediff( dd, ISNULL( auditime, a.createTime ), getdate( ) ) <= { diff_days_start } 
	AND datediff( dd, ISNULL( auditime, a.createTime ), getdate( ) ) >= { diff_days_end }
	AND d.ActivityCWTypeNo in (1,2)
	AND agentname NOT LIKE '%%测试%%' 
	and c.UnionNumber not in ('CM00006317','CM00007650')
	AND Item = '货物预存款'
	AND agentwechat NOT IN ( 'Power', 'manna24', '13076952970', 'zhulei666', 'test05' )
    and iif (a.PaymentAccount = '','无账号',a.PaymentAccount) != '无账号'
    '''
    balance_apply_data = pd.read_sql(balance_apply_SQL, erp_con)
    return balance_apply_data

In [55]:
yx_con, server = connect('yx')

In [56]:
erp_con = connect('new_ERP')
bi_con = connect('bidata')

In [57]:
yx_balance_apply_data = load_yx_balance_apply_data(2, 2, yx_con)

In [58]:
erp_balance_apply_data = load_erp_balance_apply_data(2, 2, erp_con)

In [59]:
data = pd.merge(yx_balance_apply_data[yx_balance_apply_data.SourceNo.notnull()], erp_balance_apply_data[erp_balance_apply_data.SourceNo.notnull()], how='outer', left_on=['SourceNo', 'UnionNumber','create_date','policy_name'],right_on=['SourceNo', 'UnionNumber','audi_date','zcname'],suffixes=('_yx', '_erp'))

In [60]:
data.head()

,id,trans_time_yx,create_date,SourceNo,user_id,UnionNumber,name,activityId,policy_name,title,body,amt_in,amt_out,money_yx,trans_time_erp,audi_date,realname,zcname,item,InValue,OutValue,money_erp
0,41700254,2021-04-21 09:15:28,2021-04-21,SCBA202104200000009,5037230,CM00000943,张春平,70b03f25-9ed1-4348-9006-a1ab9845781f,全品牌-2021开门红家电补差款,余额申请单,余额申请单,0.0,-800.0,-800.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,41703518,2021-04-21 10:10:22,2021-04-21,SCBA202104200000096,5078366,CM00004808,秦海霞,95b7cd83-df45-42b5-b064-21d9cf399b2e,全品牌-2021开门红混拿,余额申请单,余额申请单,0.0,-6000.0,-6000.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,41703556,2021-04-21 10:10:41,2021-04-21,SCBA202104200000095,5078366,CM00004808,秦海霞,95b7cd83-df45-42b5-b064-21d9cf399b2e,全品牌-2021开门红混拿,余额申请单,余额申请单,0.0,-4000.0,-4000.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,41704416,2021-04-21 10:22:12,2021-04-21,SCBA202104200000098,5074700,CM57583789,刘欣雨,95b7cd83-df45-42b5-b064-21d9cf399b2e,全品牌-2021开门红混拿,余额申请单,余额申请单,0.0,-5000.0,-5000.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,41705887,2021-04-21 10:35:39,2021-04-21,SCBA202104210000011,5062743,CM00000382,崔娜,95b7cd83-df45-42b5-b064-21d9cf399b2e,全品牌-2021开门红混拿,余额申请单,余额申请单,0.0,-40000.0,-40000.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
pd.set_option('display.max_columns', None)

In [62]:
data.head()

,id,trans_time_yx,create_date,SourceNo,user_id,UnionNumber,name,activityId,policy_name,title,body,amt_in,amt_out,money_yx,trans_time_erp,audi_date,realname,zcname,item,InValue,OutValue,money_erp
0,41700254,2021-04-21 09:15:28,2021-04-21,SCBA202104200000009,5037230,CM00000943,张春平,70b03f25-9ed1-4348-9006-a1ab9845781f,全品牌-2021开门红家电补差款,余额申请单,余额申请单,0.0,-800.0,-800.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,41703518,2021-04-21 10:10:22,2021-04-21,SCBA202104200000096,5078366,CM00004808,秦海霞,95b7cd83-df45-42b5-b064-21d9cf399b2e,全品牌-2021开门红混拿,余额申请单,余额申请单,0.0,-6000.0,-6000.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,41703556,2021-04-21 10:10:41,2021-04-21,SCBA202104200000095,5078366,CM00004808,秦海霞,95b7cd83-df45-42b5-b064-21d9cf399b2e,全品牌-2021开门红混拿,余额申请单,余额申请单,0.0,-4000.0,-4000.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,41704416,2021-04-21 10:22:12,2021-04-21,SCBA202104200000098,5074700,CM57583789,刘欣雨,95b7cd83-df45-42b5-b064-21d9cf399b2e,全品牌-2021开门红混拿,余额申请单,余额申请单,0.0,-5000.0,-5000.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,41705887,2021-04-21 10:35:39,2021-04-21,SCBA202104210000011,5062743,CM00000382,崔娜,95b7cd83-df45-42b5-b064-21d9cf399b2e,全品牌-2021开门红混拿,余额申请单,余额申请单,0.0,-40000.0,-40000.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
data.dtypes

id                        object
trans_time_yx     datetime64[ns]
create_date               object
SourceNo                  object
user_id                   object
UnionNumber               object
name                      object
activityId                object
policy_name               object
title                     object
body                      object
amt_in                   float64
amt_out                  float64
money_yx                 float64
trans_time_erp    datetime64[ns]
audi_date                 object
realname                  object
zcname                    object
item                      object
InValue                  float64
OutValue                 float64
money_erp                float64
dtype: object

In [77]:
dict = data.dtypes.to_dict()

In [86]:
for k in dict:
    print(k)
    print(dict[k])

id
object
trans_time_yx
datetime64[ns]
create_date
object
SourceNo
object
user_id
object
UnionNumber
object
name
object
activityId
object
policy_name
object
title
object
body
object
amt_in
float64
amt_out
float64
money_yx
float64
trans_time_erp
datetime64[ns]
audi_date
object
realname
object
zcname
object
item
object
InValue
float64
OutValue
float64
money_erp
float64


In [78]:
data.dtypes.to_dict().items()

dict_items([('id', dtype('O')), ('trans_time_yx', dtype('<M8[ns]')), ('create_date', dtype('O')), ('SourceNo', dtype('O')), ('user_id', dtype('O')), ('UnionNumber', dtype('O')), ('name', dtype('O')), ('activityId', dtype('O')), ('policy_name', dtype('O')), ('title', dtype('O')), ('body', dtype('O')), ('amt_in', dtype('float64')), ('amt_out', dtype('float64')), ('money_yx', dtype('float64')), ('trans_time_erp', dtype('<M8[ns]')), ('audi_date', dtype('O')), ('realname', dtype('O')), ('zcname', dtype('O')), ('item', dtype('O')), ('InValue', dtype('float64')), ('OutValue', dtype('float64')), ('money_erp', dtype('float64'))])

In [79]:
items = data.dtypes.to_dict().items()

In [88]:
for v,k in items:
    print(v,k)

id object
trans_time_yx datetime64[ns]
create_date object
SourceNo object
user_id object
UnionNumber object
name object
activityId object
policy_name object
title object
body object
amt_in float64
amt_out float64
money_yx float64
trans_time_erp datetime64[ns]
audi_date object
realname object
zcname object
item object
InValue float64
OutValue float64
money_erp float64
